### EXP4. 작사가 인공지능 만들기

In [1]:
import os, re 
import numpy as np
import tensorflow as tf
import glob

txt_file_path = os.getenv('HOME')+'/aiffel/AI_study/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["I'll undress you, 'cause you're tired", 'Cover you as you desire', 'When you fall asleep inside my arms']


### 데이터 전처리하기
지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로,   
문장을 토큰화 했을 때 토큰의 개수가 15개 넘어가면 데이터에서 제외하기

In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

I'll undress you, 'cause you're tired
Cover you as you desire
When you fall asleep inside my arms
May not have the fancy things
But I'll give you everything
You could ever want, it's in my arms So baby tell me yes
And I will give you everything
So baby tell me yes
And I will be all yours tonight
So baby tell me yes


In [3]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [4]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    if len(sentence.split(' ')) > 13 : continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)

# 정제된 결과를 10개만 확인해보죠
print(len(corpus))
corpus[:10]

162813


['<start> i ll undress you , cause you re tired <end>',
 '<start> cover you as you desire <end>',
 '<start> when you fall asleep inside my arms <end>',
 '<start> may not have the fancy things <end>',
 '<start> but i ll give you everything <end>',
 '<start> you could ever want , it s in my arms so baby tell me yes <end>',
 '<start> and i will give you everything <end>',
 '<start> so baby tell me yes <end>',
 '<start> and i will be all yours tonight <end>',
 '<start> so baby tell me yes <end>']

In [5]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  
    print(len(tensor))
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

162813
[[   2    5   61 ...    0    0    0]
 [   2 1108    7 ...    0    0    0]
 [   2   47    7 ...    0    0    0]
 ...
 [   2    7  455 ...    0    0    0]
 [   2   30  122 ...    0    0    0]
 [   2    5  348 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f208d64f880>


In [6]:
print(tensor[:3, :])

[[   2    5   61 2839    7    4   66    7   52  644    3    0    0    0
     0]
 [   2 1108    7   83    7 1149    3    0    0    0    0    0    0    0
     0]
 [   2   47    7  293 1579  278   13  458    3    0    0    0    0    0
     0]]


In [7]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [8]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2    5   61 2839    7    4   66    7   52  644    3    0    0    0]
[   5   61 2839    7    4   66    7   52  644    3    0    0    0    0]


In [9]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
print(len(dataset))
dataset

635


<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

### 데이터셋 분리하기
사이킷런의 train_test_split() 이용해서 훈련 데이터와 평가 데이터 분리하기   
단어장의 크기는 12,000 이상, 총 데이터의 20%를 평가 데이터로 하기

In [10]:
from sklearn.model_selection import train_test_split
# BUFFER_SIZE = len(src_input)
# BATCH_SIZE = 256
# steps_per_epoch = len(src_input) // BATCH_SIZE

# tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
# VOCAB_SIZE = tokenizer.num_words + 1   

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=7)

print("Source Train:", enc_train.shape)
print("Source val:", enc_val.shape)
print("Target Train:", dec_train.shape)
print("Target val:", dec_val.shape)

Source Train: (130250, 14)
Source val: (32563, 14)
Target Train: (130250, 14)
Target val: (32563, 14)


### 인공지능 만들기
Embedding size와 Hidden size를 조절하여   
10 Epoch안에 val_loss 값을 2.2 수준을 목표로 모델 설계하기

In [11]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)

        return out

embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [12]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-3.0227020e-05,  5.6835092e-05, -2.4579413e-04, ...,
          5.9979058e-05, -1.9626389e-04, -5.8214282e-05],
        [-2.7545591e-04,  1.1805018e-04, -5.3624628e-04, ...,
         -2.7901506e-06, -4.0213793e-04,  9.2801616e-05],
        [-6.0318474e-04,  7.0366266e-05, -5.7754363e-04, ...,
         -2.5916076e-04, -6.3087820e-04,  1.2652687e-04],
        ...,
        [-8.3530910e-04, -5.0374563e-04, -1.0701595e-03, ...,
         -6.5769121e-04, -1.8182097e-03,  1.2505852e-03],
        [-7.6403614e-04, -5.3475180e-04, -1.0293559e-03, ...,
         -5.6607829e-04, -2.0043696e-03,  1.4051079e-03],
        [-4.3865090e-04, -4.3662518e-04, -9.6763327e-04, ...,
         -4.3106062e-04, -1.9524440e-03,  1.4741145e-03]],

       [[ 3.7636983e-05,  1.5292567e-04, -3.2377109e-05, ...,
         -2.9658669e-04, -1.2614940e-04,  1.5877520e-05],
        [-4.6237386e-05,  4.3518521e-04, -4.3804664e-04, ...,
         -2.3044601e-04, 

In [13]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [14]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
635/635 [==============================] - 107s 164ms/step - loss: 3.5104
Epoch 2/10
635/635 [==============================] - 106s 167ms/step - loss: 3.0458
Epoch 3/10
635/635 [==============================] - 106s 167ms/step - loss: 2.8626
Epoch 4/10
635/635 [==============================] - 106s 167ms/step - loss: 2.7257
Epoch 5/10
635/635 [==============================] - 106s 166ms/step - loss: 2.6087
Epoch 6/10
635/635 [==============================] - 106s 167ms/step - loss: 2.5044
Epoch 7/10
635/635 [==============================] - 106s 167ms/step - loss: 2.4084
Epoch 8/10
635/635 [==============================] - 106s 167ms/step - loss: 2.3239
Epoch 9/10
635/635 [==============================] - 106s 167ms/step - loss: 2.2384
Epoch 10/10
635/635 [==============================] - 106s 167ms/step - loss: 2.1614


In [15]:
#Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [16]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [17]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i love you <end> '

### 최종
모델이 생성한 가사 한 줄은?

In [18]:
'<start> i love you , i love you <end> '

'<start> i love you , i love you <end> '